In [1]:
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup as soup
# from pandas import DataFrame
import requests
import json
import re
import sys

In [2]:
#Prep for looping over all pages.
url = "https://carriere.nrc.nl/vacatures/zoeken?&page=1"

uClient = uReq(url)
page_html = uClient.read()
uClient.close()

page_soup1 = soup(page_html,"html.parser")

end_content = page_soup1.find("ul",{"class":"pagination pagination"})

#find last page
children = end_content.findChildren()
last = int(children[14].text)
last = last+1

#Storage for all links
links1 = []

i=0
while True:
    
    i=i+1
    if i == (last+1):
        break
    url = "https://carriere.nrc.nl/vacatures/zoeken?&page=" +str(i)
    print("page: " + str(i) + " "+ str(url))
    
    uClient = uReq(url) 
    page_html = uClient.read()
    uClient.close()

    page_soup1 = soup(page_html,"html.parser")
    cont_blocks = page_soup1.findAll("div",{"class":"content"})
    for cont_block in cont_blocks:
        link = cont_block.a.get('href')
        links1.append(link)
        
print("DONE! STAGE1! ALL LINKS RETRIEVED")
print(len(links1), " Links in total")

page: 1 https://carriere.nrc.nl/vacatures/zoeken?&page=1
page: 2 https://carriere.nrc.nl/vacatures/zoeken?&page=2
page: 3 https://carriere.nrc.nl/vacatures/zoeken?&page=3
page: 4 https://carriere.nrc.nl/vacatures/zoeken?&page=4
page: 5 https://carriere.nrc.nl/vacatures/zoeken?&page=5
page: 6 https://carriere.nrc.nl/vacatures/zoeken?&page=6
page: 7 https://carriere.nrc.nl/vacatures/zoeken?&page=7
page: 8 https://carriere.nrc.nl/vacatures/zoeken?&page=8
page: 9 https://carriere.nrc.nl/vacatures/zoeken?&page=9
page: 10 https://carriere.nrc.nl/vacatures/zoeken?&page=10
page: 11 https://carriere.nrc.nl/vacatures/zoeken?&page=11
page: 12 https://carriere.nrc.nl/vacatures/zoeken?&page=12
page: 13 https://carriere.nrc.nl/vacatures/zoeken?&page=13
page: 14 https://carriere.nrc.nl/vacatures/zoeken?&page=14
page: 15 https://carriere.nrc.nl/vacatures/zoeken?&page=15
page: 16 https://carriere.nrc.nl/vacatures/zoeken?&page=16
page: 17 https://carriere.nrc.nl/vacatures/zoeken?&page=17
page: 18 https:

In [23]:
links1 

['/vacatures/dreef-beheer-bv-via-yer/senior-controller-vastgoed-98deb',
 '/vacatures/power-packer-europe-bv-via-yer/key-account-manager-automotive-cb74e',
 '/vacatures/stevin-technology-consultants-bv-via-yer/managing-consultant-6856a']

In [17]:

topics_dict = { "Job_title":[], \
                "Company":[], \
                "Location":[], \
                "Offer":[], \
               "Link":[], \
                "Posted":[], \
                "Text":[], \
                }

cc = 0
all_text = []
for links in links1:
    try:
        cc = cc + 1
        url = "https://carriere.nrc.nl" + str(links)
        print("Doing: ", url)
        uClient = uReq(url) 
        page_html = uClient.read()
        uClient.close()

        page_soup1 = soup(page_html,"html5lib")

        data_json = page_soup1.find("script", type="application/ld+json").text
        data_json = ''.join(re.sub(r"(\n|\t)*", "", data_json).split()) # --->>> REMOVE NEW LINE CHARS AND WHITESPACES

        # --->>> PREPROCESSING DATA TO REPLACE HTML CODES WITH SIGNS
        data_json = re.sub(r"\&amp;", "&", data_json) # --->>> AMPERSAND '&'
        data_json = re.sub(r"\&reg;", "®", data_json) # --->>> REGISTERED SIGN '®'

        # --->>> IMPORTANT <<<--- #
        # --->>> APPARENTLY SOUP MAKES SOMETIMES A MISTAKE WITH WRITING ANOTHER COMMA AT THE END OF A BLOCK
        # --->>> WHERE NO MORE BLOCKS ARE COMING AND JSON DOES NOT LIKE THAT...
        data_json = re.sub(r"\},\s\}", "} }", data_json) # --->>> REMOVE TRAILING COMMA IF NO MORE BLOCKS ARE COMING

        try:
            newDictionary = json.loads(data_json) # --->>> LOAD EVERYTHING TO A DICTIONARY
        except (Exception, ArithmeticError) as e:
            template = "An exception of type {0} occurred. Arguments:\n{1!r}" # --->>> ERROR MESSAGE TEMPLATE
            message = template.format(type(e).__name__, e.args) # --->>> COMPILE ERROR MESSAGE
            print (message) # --->>> PRINT THE ERROR
            print(data_json) # --->>> PRINT DATA TO CHECK WHAT COULD CAUSE ERROR
            sys.exit("JSONDecodeError") # --->>> STOP EXECUTION OF SCRIPT

        title = newDictionary['title']
        company = newDictionary['hiringOrganization']['name']
        print(company)
        offer = newDictionary['@type']
        location = newDictionary['jobLocation']['address']['addressLocality']
        date = newDictionary['datePosted']

        #preparing for CSV
        title.replace(",",";")
        company.replace(",",";")
        offer.replace(",",";")
        location.replace(",",";")
        date.replace(",",";") 

        #Writing to Pandas frame
        topics_dict["Job_title"].append(title)
        topics_dict["Company"].append(company)
        topics_dict["Location"].append(location)
        topics_dict["Offer"].append(offer)
        topics_dict["Link"].append(url)
        topics_dict["Posted"].append(date)

        #OBTAIN TEXT DATA
        #I USED CHILDREN
        vcs = page_soup1.find("div",{"class":"js-register-outgoing-clicks"})
        tekst = vcs.text

        tekst2 = re.sub(r"[\n\t]*", "", tekst)

        str_text = ''.join(tekst2) 
        str_text = re.sub(r'[\n\t]',' ', str_text) 

        # --->>> TRY TO WRITE EVERYTHING TO A NEW LINE IN THE CSV FILE
        # --->>> CHANGE THE TYPE OF CERTAIN ELEMENTS IN CASE THERE WAS AN EXCEPTION DURING RUNTIME
        try:
            print('no error')
    #             ff.write(
    #                 title.replace(",",";") + "," +
    #                 company.replace(",",";") + "," +
    #                 location.replace(",",";") + "," + 
    #                 offer.replace(",",";") + "," +
    #                 url.replace(",",";") + "," + 
    #                 date.replace(",",";") + "," + 
    #                 str_text.replace(",",";") + "\n"
    #             )
        except (Exception, ArithmeticError) as e:
            template = "An exception of type {0} occurred. Arguments:\n{1!r}" # --->>> ERROR MESSAGE TEMPLATE
            message = template.format(type(e).__name__, e.args) # --->>> COMPILE ERROR MESSAGE
            print (message) # --->>> PRINT THE ERROR

            print(type(title), title) # --->>> PRINT THE TYPE OF THE TITLE
            print(type(company), company) # --->>> PRINT THE TYPE OF THE COMPANY
            print(type(location), location) # --->>> PRINT THE TYPE OF THE LOCATION
            print(type(offer), offer) # --->>> PRINT THE TYPE OF THE OFFER
            print(type(url), url) # --->>> PRINT THE TYPE OF THE URL
            print(type(date), date) # --->>> PRINT THE TYPE OF THE DATE
            print(type(str_text)) # --->>> PRINT THE TYPE OF THE STR_TEXT

            # --->>> CHANGE THE VALUE OF THE MISSING VARIABLE
            # --->>> SELECT SOMETHING REASONABLE TO REPLACE THE VALUE WITH
            # --->>> FOR NOW I JUST CHANGE THE TYPE OF THE VARIABLE TO STRING
            if title is None:
                # title = DEFAULT_TITLE
                title = str(title)
            if company is None:
                # company = DEFAULT_COMPANY
                company = str(company)
            if location is None:
                # location = DEFAULT_LOCATION
                location = str(location)
            if offer is None:
                # offer = DEFAULT_OFFER
                offer = str(offer)
            if date is None:
                # company = DEFAULT_DATE
                company = str(company)
            if str_text is None:
                # str_text = DEFAULT_TEXT
                str_text = str(str_text)

            # --->>> WRITE THE LINE NOW TO CSV WITH NEW VALUES
    #             ff.write(
    #                 title.replace(",",";") + "," +
    #                 company.replace(",",";") + "," +
    #                 location.replace(",",";") + "," + 
    #                 offer.replace(",",";") + "," +
    #                 url.replace(",",";") + "," + 
    #                 date.replace(",",";") + "," + 
    #                 str_text.replace(",",";") + "\n"
    #             )
        
        print("Done writing to CSV number: " + str(cc))
    except:
        pass

Doing:  https://carriere.nrc.nl/vacatures/alliander/alliander-management-traineeship-1-oktober-2018-4f5c4
Alliander
Doing:  https://carriere.nrc.nl/vacatures/yer/plant-manager-locatie-zeeland-0b298
YER
no error
Done writing to CSV number: 2
Doing:  https://carriere.nrc.nl/vacatures/dreef-beheer-bv-via-yer/senior-controller-vastgoed-98deb
DreefBeheerB.V.viaYER
no error
Done writing to CSV number: 3
Doing:  https://carriere.nrc.nl/vacatures/power-packer-europe-bv-via-yer/key-account-manager-automotive-cb74e
PowerPackerEuropeB.V.viaYER
no error
Done writing to CSV number: 4
Doing:  https://carriere.nrc.nl/vacatures/stevin-technology-consultants-bv-via-yer/managing-consultant-6856a
StevinTechnologyConsultantsB.V.viaYER
no error
Done writing to CSV number: 5
Doing:  https://carriere.nrc.nl/vacatures/leeuwendaal/afdelingshoofd-beleid-4b1eb
PensioenfederatieviaLeeuwendaal
no error
Done writing to CSV number: 6
Doing:  https://carriere.nrc.nl/vacatures/college-voor-de-rechten-van-de-mens-via-a

OpenUniversiteit
no error
Done writing to CSV number: 45
Doing:  https://carriere.nrc.nl/vacatures/universiteit-maastricht/phd-or-postdoc-position-at-the-department-of-radiology-nutrition-and-human-movement-sciences-development-and-application-of-state-of-the-art-magnetic-resonance-spectroscopy-methodology-in-clincial-metabolic-research-4a94d
UniversiteitMaastricht
no error
Done writing to CSV number: 46
Doing:  https://carriere.nrc.nl/vacatures/universiteit-van-amsterdam-uva/2-phd-candidates-in-mathematics-d63d5
UniversiteitvanAmsterdam(UvA)
no error
Done writing to CSV number: 47
Doing:  https://carriere.nrc.nl/vacatures/vu/associate-professor-of-environmental-and-resource-economics-ea847
VU
no error
Done writing to CSV number: 48
Doing:  https://carriere.nrc.nl/vacatures/universiteit-maastricht/postdoctoral-fellow-position-on-fair-principles-in-data-science-82420
UniversiteitMaastricht
no error
Done writing to CSV number: 49
Doing:  https://carriere.nrc.nl/vacatures/vu/associate-pro

UniversiteitLeiden(LEI)
no error
Done writing to CSV number: 92
Doing:  https://carriere.nrc.nl/vacatures/rijksuniversiteit-groningen-rug/assistant-professor-universitair-docent-european-culture-literature-russian-b8147
RijksuniversiteitGroningen(RUG)
no error
Done writing to CSV number: 93
Doing:  https://carriere.nrc.nl/vacatures/rijksuniversiteit-groningen-rug/assistant-professor-universitair-docent-european-culture-literature-politics-b8147
RijksuniversiteitGroningen(RUG)
no error
Done writing to CSV number: 94
Doing:  https://carriere.nrc.nl/vacatures/wageningen-ur/phd-field-and-farm-monitoring-for-sustainable-potato-production-dc112
WageningenUR
no error
Done writing to CSV number: 95
Doing:  https://carriere.nrc.nl/vacatures/rijksuniversiteit-groningen-rug/lecturers-artificial-intelligence-programme-1-6-fte-b8147
RijksuniversiteitGroningen(RUG)
no error
Done writing to CSV number: 96
Doing:  https://carriere.nrc.nl/vacatures/radboudumc/postdoc-small-rna-pathways-in-virus-transmi

Radboudumc
no error
Done writing to CSV number: 134
Doing:  https://carriere.nrc.nl/vacatures/amc/postdoc-medical-biology-d2bc2
AMC
no error
Done writing to CSV number: 135
Doing:  https://carriere.nrc.nl/vacatures/technische-universiteit-delft-tud/assistant-professor-tenure-track-modeling-and-simulation-21a20
TechnischeUniversiteitDelft(TUD)
no error
Done writing to CSV number: 136
Doing:  https://carriere.nrc.nl/vacatures/technische-universiteit-delft-tud/simulation-engineer-21a20
TechnischeUniversiteitDelft(TUD)
no error
Done writing to CSV number: 137
Doing:  https://carriere.nrc.nl/vacatures/technische-universiteit-delft-tud/tenure-track-position-assistant-professor-internet-of-things-and-ubiquitous-computing-61177
TechnischeUniversiteitDelft(TUD)
no error
Done writing to CSV number: 138
Doing:  https://carriere.nrc.nl/vacatures/universiteit-maastricht/postdoc-senior-researcher-fhml-school-for-cardiovascular-diseases-carim-department-of-physiology-47e87
UniversiteitMaastricht
no e

TechnischeUniversiteitDelft(TUD)
no error
Done writing to CSV number: 179
Doing:  https://carriere.nrc.nl/vacatures/technische-universiteit-eindhoven/phd-student-in-applied-analysis-2-x-1-0-fte-470ea
TechnischeUniversiteitEindhoven
no error
Done writing to CSV number: 180
Doing:  https://carriere.nrc.nl/vacatures/universiteit-utrecht-uu/research-assistant-social-change-for-sustainability-1-0-fte-e553d
UniversiteitUtrecht(UU)
no error
Done writing to CSV number: 181
Doing:  https://carriere.nrc.nl/vacatures/deltion-college/leraar-apotheker-0f754
DeltionCollege
no error
Done writing to CSV number: 182
Doing:  https://carriere.nrc.nl/vacatures/hoogheemraadschap-de-stichtse-rijnlanden/adviseur-financien-en-rapportage-7d752
HoogheemraadschapDeStichtseRijnlanden
no error
Done writing to CSV number: 183
Doing:  https://carriere.nrc.nl/vacatures/kruger/financieel-manager-7b898
Kruger
no error
Done writing to CSV number: 184
Doing:  https://carriere.nrc.nl/vacatures/universiteit-maastricht/post

...

TechnischeUniversiteitDelft(TUD)
no error
Done writing to CSV number: 490
Doing:  https://carriere.nrc.nl/vacatures/technische-universiteit-delft-tud/phd-precision-motion-control-for-mechatronic-systems-eb3db
TechnischeUniversiteitDelft(TUD)
no error
Done writing to CSV number: 491
Doing:  https://carriere.nrc.nl/vacatures/luke/production-manager-1e3c9
Na-Nomi
no error
Done writing to CSV number: 492
Doing:  https://carriere.nrc.nl/vacatures/luke/operations-manager-60207
DPD
no error
Done writing to CSV number: 493
Doing:  https://carriere.nrc.nl/vacatures/boelen-adviesgroep-b-v/algemeen-directeur-0ef06
BoelenAdviesgroepB.V.
Doing:  https://carriere.nrc.nl/vacatures/synerlogic-bv-via-yer/production-manager-6392c
SynerlogicB.V.viaYER
no error
Done writing to CSV number: 495
Doing:  https://carriere.nrc.nl/vacatures/akzonobel/organizational-effectiveness-manager-07352
AkzoNobel
no error
Done writing to CSV number: 496
Doing:  https://carriere.nrc.nl/vacatures/gemeente-nissewaard/integraa

TechnischeUniversiteitEindhoven
no error
Done writing to CSV number: 537
Doing:  https://carriere.nrc.nl/vacatures/centrum-wiskunde-informatica/phd-student-on-the-subject-of-modelling-and-predicting-multiphase-flow-using-uncertainty-quantification-and-data-assimilation-caf52
CentrumWiskunde&Informatica
no error
Done writing to CSV number: 538
Doing:  https://carriere.nrc.nl/vacatures/open-universiteit/promovendus-chronic-and-acute-stressors-in-high-risk-organizations-1-0-fte-f9281
OpenUniversiteit
no error
Done writing to CSV number: 539
Doing:  https://carriere.nrc.nl/vacatures/universiteit-van-amsterdam-uva/two-faculty-positions-in-condensed-matter-physics-with-focus-on-energy-and-quantum-materials-0b61a
UniversiteitvanAmsterdam(UvA)
no error
Done writing to CSV number: 540
Doing:  https://carriere.nrc.nl/vacatures/postnl/it-service-management-coordinator-9d32d
PostNL
no error
Done writing to CSV number: 541
Doing:  https://carriere.nrc.nl/vacatures/embrace/chief-technology-officer-c

WageningenUR
no error
Done writing to CSV number: 593
Doing:  https://carriere.nrc.nl/vacatures/technische-universiteit-eindhoven/postdoc-position-for-1-year-inspec-ccd10
TechnischeUniversiteitEindhoven
no error
Done writing to CSV number: 594
Doing:  https://carriere.nrc.nl/vacatures/technische-universiteit-eindhoven/phd-integrating-chiral-magnetic-domain-wall-motion-with-photonics-4d9ea
TechnischeUniversiteitEindhoven
no error
Done writing to CSV number: 595
Doing:  https://carriere.nrc.nl/vacatures/technische-universiteit-delft-tud/phd-through-process-modelling-for-steel-product-development-ce060
TechnischeUniversiteitDelft(TUD)
no error
Done writing to CSV number: 596
Doing:  https://carriere.nrc.nl/vacatures/kcap-architectsplanners/managing-director-kcap-architects-planners-691d4
KCAPArchitects&Planners
no error
Done writing to CSV number: 597
Doing:  https://carriere.nrc.nl/vacatures/bis-bedrijfs-informatie-systemen-bv/medewerker-inkoop-procurement-officer-6edf8
BisBedrijfsInform

HorizonJeugdzorgenOnderwijs
no error
Done writing to CSV number: 637
Doing:  https://carriere.nrc.nl/vacatures/rijksuniversiteit-groningen-rug/postdoc-human-ex-vivo-model-to-study-nonalcoholic-fatty-liver-disease-1-0-fte-753c9
RijksuniversiteitGroningen(RUG)
no error
Done writing to CSV number: 638
Doing:  https://carriere.nrc.nl/vacatures/universiteit-utrecht-uu/postdoctoral-fellow-assistant-professor-position-in-host-pathogen-interaction-research-1-0-fte-029bf
UniversiteitUtrecht(UU)
no error
Done writing to CSV number: 639
Doing:  https://carriere.nrc.nl/vacatures/technische-universiteit-delft-tud/phd-identification-of-lax-integrable-systems-9b574
TechnischeUniversiteitDelft(TUD)
no error
Done writing to CSV number: 640
Doing:  https://carriere.nrc.nl/vacatures/technische-universiteit-delft-tud/phd-complex-network-games-0807d
TechnischeUniversiteitDelft(TUD)
no error
Done writing to CSV number: 641
Doing:  https://carriere.nrc.nl/vacatures/legal-people/hoofd-juridische-zaken-directi

ITILITYB.V.viaYER
no error
Done writing to CSV number: 687
Doing:  https://carriere.nrc.nl/vacatures/mercuri-urval/national-clinical-specialist-0685c
FreseniusMedicalCareviaMercuriUrval
no error
Done writing to CSV number: 688
Doing:  https://carriere.nrc.nl/vacatures/ing/hbo-wo-meewerkstage-ing-private-wealth-asset-management-te-amsterdam-d5e4d
ING
no error
Done writing to CSV number: 689
Doing:  https://carriere.nrc.nl/vacatures/mercuri-urval/hoofd-i-a-informatiemanager-71695
PrinsBernhardCultuurfondsviaMercuriUrval
no error
Done writing to CSV number: 690
Doing:  https://carriere.nrc.nl/vacatures/gemeente-dronten/teammanager-d7724
GemeenteDronten
no error
Done writing to CSV number: 691
Doing:  https://carriere.nrc.nl/vacatures/gemeente-dordrecht/externe-voorzitter-rekenkamercommissie-32cc0
GemeenteDordrecht
no error
Done writing to CSV number: 692
Doing:  https://carriere.nrc.nl/vacatures/stichting-vluchteling/ervaren-persvoorlichter-m-v-46e06
StichtingVluchteling
no error
Done wri

In [57]:

uClient = uReq(url) 
page_html = uClient.read()
uClient.close()

page_soup1 = soup(page_html,"html5lib")

data = page_soup1.find("script", type="application/ld+json").text
#data = ''.join(re.sub(r"(\n|\t)*", "", data).split()) # --->>> REMOVE NEW LINE CHARS AND WHITESPACES

# --->>> PREPROCESSING DATA TO REPLACE HTML CODES WITH SIGNS
#data = re.sub(r"\&amp;", "&", data) # --->>> AMPERSAND '&'
#data = re.sub(r"\&reg;", "®", data) # --->>> REGISTERED SIGN '®'

# --->>> IMPORTANT <<<--- #
# --->>> APPARENTLY SOUP MAKES SOMETIMES A MISTAKE WITH WRITING ANOTHER COMMA AT THE END OF A BLOCK
# --->>> WHERE NO MORE BLOCKS ARE COMING AND JSON DOES NOT LIKE THAT...
#data = re.sub(r"\},\s\}", "} }", data) # --->>> REMOVE TRAILING COMMA IF NO MORE BLOCKS ARE COMING

newDictionary = json.loads(data) # --->>> LOAD EVERYTHING TO A DICTIONARY

company = newDictionary['hiringOrganization']['name']
print(data)
print(company)


  {
  "@context": "http://schema.org",
  "@type": "JobPosting",
  "baseSalary": {
    "@type": "MonetaryAmount",
    "minValue": null,
    "maxValue": null,
    "currency": "EUR"
  },
  "datePosted": "2018-07-20",
  "validThrough": "2018-09-17T00:00:00+02:00",
  "description": "Als Managing Consultant bij Stevin ken je de sector als geen ander. Je hebt je ideeën over de toekomst van de energiesector én de gedrevenheid en durf om deze ideeën ook waar te maken. Je bent ondernemend, je herkent en benut kansen bij klanten. Vraagstukken van de klanten zijn bijvoorbeeld: ombouw van installaties naar duurzame energiesystemen, big data toepassen om storingen te voorspellen, verbetering van de assetmanagementstrategie of uitbreiding van het hoogspanningsnet. Als operationeel leidinggevende ben je verantwoordelijk voor de projectteams en help je de consultants in hun professionele ontwikkeling. ",
  "educationRequirements": "wo",
  "employmentType": "steady",
  "experienceRequirements": "\nJe h